In [72]:
print("ok")

ok


In [73]:
%pwd

'c:\\'

In [74]:
import os 
os.chdir("../")

In [75]:
%pwd

'c:\\'

In [76]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [77]:
#Extarct Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    
    documents = loader.load()
    return documents

In [78]:
extracted_data = load_pdf_file(data=r"C:\Users\SHANTANU KADLAG\OneDrive\Desktop\medical bot\Medical-Chatbot-GenAI\Data")


In [82]:
#split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [83]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks" , len(text_chunks))

Length of Text Chunks 5859


In [84]:
from langchain.embeddings import HuggingFaceEmbeddings

In [85]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [88]:
embeddings = download_hugging_face_embeddings()

c:\Users\SHANTANU KADLAG\.conda\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SHANTANU KADLAG\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

In [89]:
query_result =embeddings.embed_query("Hello world")
print("Length" , len(query_result))

Length 384


In [ ]:
#query_result

[-0.03447727859020233,
 0.031023195013403893,
 0.0067350007593631744,
 0.02610897831618786,
 -0.03936203941702843,
 -0.16030246019363403,
 0.06692393869161606,
 -0.006441489793360233,
 -0.04745043069124222,
 0.014758850447833538,
 0.07087530940771103,
 0.05552756413817406,
 0.019193332642316818,
 -0.026251299306750298,
 -0.010109570808708668,
 -0.026940522715449333,
 0.022307388484477997,
 -0.022226659581065178,
 -0.1496925801038742,
 -0.017493056133389473,
 0.00767620699480176,
 0.05435231328010559,
 0.003254441311582923,
 0.03172589838504791,
 -0.08462132513523102,
 -0.02940601296722889,
 0.0515955425798893,
 0.048124007880687714,
 -0.003314810572192073,
 -0.05827917903661728,
 0.04196931794285774,
 0.022210605442523956,
 0.12818878889083862,
 -0.022338947281241417,
 -0.011656264774501324,
 0.06292837113142014,
 -0.03287631645798683,
 -0.09122605621814728,
 -0.031175334006547928,
 0.052699584513902664,
 0.04703482240438461,
 -0.0842030793428421,
 -0.030056186020374298,
 -0.0207448396

In [91]:
pip install pinecone


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv

load_dotenv("C:\\Users\\SHANTANU KADLAG\\OneDrive\\Desktop\\medical bot\\Medical-Chatbot-GenAI\\.env")


True

In [122]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [154]:
from pinecone.grpc import PineconeGRPC as PineconeGRPC
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-qwgft4i.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [123]:
import os 
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]= OPENAI_API_KEY

In [112]:
#Embed each chunk and upset the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [138]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk upset the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [139]:
docsearch

In [140]:
retriever =docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [141]:
retrieved_docs=retriever.invoke("What is Acne ?")

In [142]:
retrieved_docs

[Document(id='3628dd0f-0bbb-4288-8067-a42a9111ccb6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\SHANTANU KADLAG\\OneDrive\\Desktop\\medical bot\\Medical-Chatbot-GenAI\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='285ad14e-60fb-451a-92b6-8e1cea12bd78', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\SHANTANU KADLAG\\OneDrive\\Desktop\\medical bot\\Medical-Chatbot-GenAI\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vul

In [156]:
from langchain_openai import OpenAI
llm = Ollama(model="llama3", temperature=0.4, num_ctx=500)


In [149]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are en assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer ,say that you "
    "don't know . Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [150]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [153]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

According to the GALE ENCYCLOPEDIA OF MEDICINE, Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris, also known as common acne, is the most common skin disease affecting nearly 17 million people in the United States.
